In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import NMF
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel
from sklearn.metrics import mean_squared_error

In [2]:
processed_df = pd.read_pickle("data/processed_recipes.pkl")
processed_df

,name,ingredients,description,instructions,cuisine,course,diet,prep_time,categorized_prep_time,cleaned_ingredients,allergens,processed_name,processed_instructions,processed_description,allergen_type,processed_ingredients,combined_all_contexts,combined_name_ingredients
0,Doddapatre Tambuli Recipe (Karpooravalli Thayi...,"4 cups Indian borage (Doddapatre) , cleaned an...",Doddapatre Tambuli Recipe is a popular healing...,"To begin making the Doddapatre Tambuli, first ...",Indian,Side Dish,Vegetarian,30.0,~30 min,"[indian borage doddapatre, curd dahi yogurt, c...",[yogurt],doddapatre tambuli karpooravalli thayir pachadi,doddapatre tambuli doddapatre leave dry towel ...,doddapatre tambuli popular healing cold yoghur...,[Dairy],indian borage doddapatre curd dahi yogurt cumi...,doddapatre tambuli karpooravalli thayir pachad...,doddapatre tambuli karpooravalli thayir pachad...
1,Fish Tandoori Recipe,"1 Fish , nicely washed with bones on (I used P...",Have you ever tried out fish tandoori? Trust m...,"To begin making the Fish Tandoori recipe, keep...",North Indian,Appetizer,High Protein Non Vegetarian,80.0,~1 hour 30 min,"[fish, curd dahi yogurt, onion, green chilli, ...",[fish],fish tandoori,fish tandoori head fin inside fish many horizo...,fish tandoori delicious chicken tandoori tando...,[Fish and Shellfish],fish curd dahi yogurt onion green chilli tando...,fish tandoori fish curd dahi yogurt onion gree...,fish tandoori fish curd dahi yogurt onion gree...
2,Arbi Shimla Mirch Sabzi Recipe – Colocasia Cap...,300 grams Colocasia root (Arbi) 1 Green Bell ...,Arbi Shimla Mirch Sabzi Recipe is a variation ...,To begin making the Arbi Shimla Mirch Sabzi Re...,North Indian,Main Course,No Onion No Garlic (Sattvic),55.0,~1 hour,"[colocasia root arbi, green bell pepper capsic...",[peanut],arbi shimla mirch sabzi colocasia capsicum sabzi,arbi shimla mirch sabzi arbi pressure pressure...,arbi shimla mirch sabzi variation aloo capsicu...,"[Tree Nuts and Peanuts, Legumes and Seeds]",colocasia root arbi green bell pepper capsicum...,arbi shimla mirch sabzi colocasia capsicum sab...,arbi shimla mirch sabzi colocasia capsicum sab...
3,Ambur Style Brinjal Curry Recipe,5 Brinjal (Baingan / Eggplant) 1 Onion 2 Tom...,Brinjal Curry is prepared in different ways in...,To begin making the Ambur Style Brinjal Curry ...,Indian,Lunch,Vegetarian,20.0,~30 min,"[brinjal baingan eggplant, onion, ginger garli...","[sesame, peanut, mustard]",ambur style brinjal curry,ambur style brinjal curry oil heavy bottomed o...,brinjal curry different way different region s...,"[Legumes and Seeds, Tree Nuts and Peanuts]",brinjal baingan eggplant onion ginger garlic p...,ambur style brinjal curry brinjal baingan eggp...,ambur style brinjal curry brinjal baingan eggp...
4,Mavinakayi Menasinakai Curry Recipe - Raw Man...,"1 cup Mango (Raw) , diced 2 tablespoons Jagge...",Mangalorean Mavinakayi Menasinakai Curry Recip...,To begin making Mangalorean Mavinakayi Menasin...,Mangalorean,Side Dish,Vegetarian,35.0,~45 min,"[mango raw, jaggery, turmeric powder haldi, ta...",[sesame],mavinakayi menasinakai curry raw mango coconut...,mangalorean mavinakayi menasinakai curry raw m...,mangalorean mavinakayi menasinakai curry raw m...,[Legumes and Seeds],mango raw jaggery turmeric powder haldi tamari...,mavinakayi menasinakai curry raw mango coconut...,mavinakayi menasinakai curry raw mango coconut...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9491,Spinach Oats Vada Recipe,"Spinach Leaves (Palak), Chana dal (Bengal Gram...",Spinach Oats Vada Recipe is a healthy twist to...,"To prepare Spinach Oats Vada Recipe, in a larg...",South Indian,Snack,Vegetarian,50.0,~1 hour,"[spinach leave palak, chana dal bengal gram da...",[oats],spinach oats vada,spinach oats vada large mixing ingredient vada...,spinach oats vada healthy twist vada superfood...,[Grains and Gluten],spinach leave palak chana dal bengal gram dal ...,spinach oats vada spinach leave palak chana da...,sp

In [3]:
tfidf_vectorizer = TfidfVectorizer()
count_vectorizer = CountVectorizer()

Find dish name in recipes and filter them based on user preferences

In [4]:
def find_filter_recipes (dish_name, df, ingredients=None, cuisine=None, course=None, diet=None, prep_time=None, allergen_type=None): 
  
    filtered_df = df[df['name'].str.contains(dish_name, case=False, na=False)]
    print ((f"Number of found dishes before filtering : {filtered_df.shape[0]}"))

    if ingredients:
        # Check if all specified ingredients are present in the cleaned_ingredients list
        filtered_df = filtered_df[filtered_df['cleaned_ingredients'].apply(lambda x: all(ing in x for ing in ingredients))]            
    if cuisine:
        filtered_df = filtered_df[filtered_df['cuisine'] == cuisine]
    if course:
        filtered_df = filtered_df[filtered_df['course'] == course]
    if diet:
        filtered_df = filtered_df[filtered_df['diet'] == diet]
    if prep_time:
        filtered_df = filtered_df[filtered_df['categorized_prep_time'] == prep_time]
    if allergen_type:
        # Exclude recipes with allergens in user preference
        filtered_df = filtered_df[~filtered_df['allergen_type'].apply(lambda x: bool(set(x) & set(allergen_type)))]

    if filtered_df.empty:
        print("No recipes found matching the criteria.")
        return None
    return filtered_df

In [5]:
filtered_df = find_filter_recipes ("masala dosa", processed_df) ####ingredients = ["ghee", "idli rice"], diet = "Vegetarian")
filtered_df

Number of found dishes before filtering : 26


,name,ingredients,description,instructions,cuisine,course,diet,prep_time,categorized_prep_time,cleaned_ingredients,allergens,processed_name,processed_instructions,processed_description,allergen_type,processed_ingredients,combined_all_contexts,combined_name_ingredients
746,Mysore Masala Dosa Made With Ragi Dosa Batter,"For the Dosa Batter 1 cup Idli Rice , (a speci...",Mysore Masala Dosa Recipe is a famous breakfas...,"To begin making the Mysore Masala Dosa Recipe,...",Karnataka,South Indian Breakfast,Vegetarian,60.0,~1 hour,"[idli rice, special rice available indian stor...",[mustard],mysore masala dosa ragi dosa batter,mysore masala dosa ragi millet idli dosa batte...,mysore masala dosa famous dish among southern ...,[Legumes and Seeds],idli rice special rice available indian store ...,mysore masala dosa ragi dosa batter idli rice ...,mysore masala dosa ragi dosa batter idli rice ...
1005,Mushroom Corn Masala Dosa Recipe Using Ragi Id...,To make the Ragi Dosa Batter 2 cups White Urad...,Mushroom Corn Masala Dosa Recipe Using Ragi Id...,To begin making the Mushroom Corn Masala Dosa ...,Fusion,South Indian Breakfast,Vegetarian,35.0,~45 min,"[white urad dal whole, methi seed fenugreek se...",unknown,mushroom corn masala dosa ragi idli dosa batter,mushroom corn masala dosa ragi idli dosa batte...,mushroom corn masala dosa ragi idli dosa batte...,Unknown,white urad dal whole methi seed fenugreek seed...,mushroom corn masala dosa ragi idli dosa batte...,mushroom corn masala dosa ragi idli dosa batte...
1007,Kanchipuram Masala Dosa Recipe With Soya Dosa ...,For Soya Dosa Batter 2 cups Idli Rice 1 cup S...,Kanchipuram Masala Dosa is a breakfast recipe ...,"To begin making Kanchipuram Masala Dosa, first...",South Indian,South Indian Breakfast,Vegetarian,390.0,~6.0 hours 30 min,"[idli rice, soybean whole soya dal, white urad...","[soy, mustard, soya]",kanchipuram masala dosa soya dosa batter,kanchipuram masala dosa soya dosa batter regul...,kanchipuram masala dosa beetroot carrot onion ...,[Legumes and Seeds],idli rice soybean whole soya dal white urad da...,kanchipuram masala dosa soya dosa batter idli ...,kanchipuram masala dosa soya dosa batter idli ...
1189,Schezwan Masala Dosa Recipe Using Classic Idli...,For Classic Dosa Batter 1 1/4 cup White Urad D...,'Dosa' and 'Noodles' are two very popular stre...,To begin making the Schezwan Masala Dosa Recip...,Fusion,Snack,Vegetarian,70.0,~1 hour 15 min,"[white urad dal whole, methi seed fenugreek se...",unknown,schezwan masala dosa classic idli dosa batter,schezwan masala dosa dosa batter classic idli ...,dosa noodle popular street dish india schezwan...,Unknown,white urad dal whole methi seed fenugreek seed...,schezwan masala dosa classic idli dosa batter ...,schezwan masala dosa classic idli dosa batter ...
1221,Paneer Masala Dosa Recipe-Paneer Bhurji Dosa,For Paneer Masala/ Bhurji 2 cups Paneer (Homem...,The Paneer Bhurji Dosa Recipe is also known as...,To begin making the Paneer Masala Dosa Recipe-...,South Indian,South Indian Breakfast,High Protein Vegetarian,40.0,~45 min,"[paneer homemade cottage cheese, green bell pe...","[cheese, butter, paneer]",paneer masala dosa paneer bhurji dosa,paneer masala dosa paneer bhurji dosa prep ing...,paneer bhurji dosa paneer masala dosa deliciou...,[Dairy],paneer homemade cottage cheese green bell pepp...,paneer masala dosa paneer bhurji dosa paneer h...,paneer masala dosa paneer bhurji dosa paneer h...
1231,Cheesy Mushroom Masala Dosa Recipe,For ragi dosa batter 1 cup Idli Rice 2 cups R...,Cheesy Mushroom Masala Dosa Recipe is a wholes...,To begin making the Cheesy Mushroom Masala Dos...,South Indian,South Indian Breakfast,Vegetarian,330.0,~5.0 hours 30 min,"[idli rice, ragi seed, barley seed, white urad...","[cheese, barley, paneer]",cheesy mushroom masala dosa,cheesy mushroom masala dosa ragi dosa ragi dos...,cheesy mushroom masala dosa wholesome dish fam...,"[Dairy, Grains and Gluten]",idli rice ragi seed barley seed white urad dal...,cheesy 

Calculate the best number of topics using coherence score for the filtered dataframe

In [6]:
def compute_coherence_scores(filtered_df, vectorizer, model, min_topics=2, max_topics_limit = 10 ):
    coherence_scores = []
    
    # Transform texts to TF-IDF matrix
    vectorizer_matrix = vectorizer.fit_transform(filtered_df["combined_name_ingredients"])
    
    # Tokenize each text in `texts` for Gensim's coherence calculation
    tokenized_texts = [text.split() for text in filtered_df["combined_name_ingredients"]]
    dictionary = Dictionary(tokenized_texts)
    corpus = [dictionary.doc2bow(text) for text in tokenized_texts]

    max_topics = min(max_topics_limit, vectorizer_matrix.shape[0])
    num_topics_range = range(min_topics, max_topics + 1)
        
    # Iterate over each specified topic count
    for num_topics in num_topics_range :
        called_model = model(n_components=num_topics, random_state=42)
        called_matrix = called_model.fit_transform(vectorizer_matrix)
        
        # Extract top words for each topic and format as list of tokens
        feature_names = vectorizer.get_feature_names_out()
        topics = [[feature_names[i] for i in topic.argsort()[:-11:-1]] for topic in called_model.components_]
        
        tokenized_topics = [[word for word in topic] for topic in topics]# Coherence model requires a list of tokenized topics

        coherence_model = CoherenceModel(
            topics=tokenized_topics, 
            texts=tokenized_texts, 
            dictionary=dictionary, 
            coherence='c_v'
        )
        coherence_score = coherence_model.get_coherence()
        coherence_scores.append(coherence_score)
        
        #print(f"Number of Topics: {num_topics}, Coherence Score: {coherence_score}")

    # Find the best number of topics
    best_num_topics = num_topics_range[coherence_scores.index(max(coherence_scores))]
    print(f"Best Number of Topics: {best_num_topics} with Coherence Score: {max(coherence_scores)}")

    return best_num_topics

Train the recommendation model using topic models and vectorizers

In [ ]:
def get_recommendations(filtered_df, vectorizer, model, num_recommendations=5): 

    best_num_topics = compute_coherence_scores(filtered_df, vectorizer, model)

    vectorizer_matrix = vectorizer.fit_transform(filtered_df['combined_name_ingredients'])

    # Ensure all values in tfidf_matrix are non-negative and finite
    vectorizer_matrix = np.nan_to_num(vectorizer_matrix.toarray(), nan=0.0, posinf=0.0, neginf=0.0)

    # Train the NMF model with the optimal number of topics    
    called_model = model(n_components=best_num_topics, random_state=42)
    called_model.fit(vectorizer_matrix)
    topic_matrix = called_model.transform(vectorizer_matrix)  # Topic distribution for each recipe

    filtered_df = filtered_df.copy()

    # Add topic distributions to the filtered DataFrame
    for i in range(best_num_topics):
        filtered_df[f'topic_{i}'] = topic_matrix[:, i]

    # Find the index of the dish within the filtered DataFrame
    dish_index = filtered_df.reset_index().index[0]  # Get the first match if there are multiple

    cosine_similarities = cosine_similarity(topic_matrix)
    
    # Get indices of the most similar recipes
    similar_indices = cosine_similarities[dish_index].argsort()[-num_recommendations-1:-1][::-1]

    # Fetch recommended recipes
    recommended_recipes = filtered_df.iloc[similar_indices].copy()
    recommended_recipes["similarity_score"] = cosine_similarities[dish_index][similar_indices]

    # Scale and format similarity scores to percentages
    recommended_recipes['similarity_score'] = (recommended_recipes['similarity_score'] * 100).round(2).astype(str) + '%'

    return recommended_recipes[['name', 'similarity_score', 'cleaned_ingredients', 'cuisine', 'course', 'diet', 'allergens', 'prep_time']]

Get recipe recommendation using the best NMF model on TF-IDF vectorizer

In [8]:
recommended_recipes_nmf_tfidf = get_recommendations(filtered_df, tfidf_vectorizer, NMF) 
recommended_recipes_nmf_tfidf

Best Number of Topics: 9 with Coherence Score: 0.7778235465876768


,name,similarity_score,cleaned_ingredients,cuisine,course,diet,allergens,prep_time
5609,Mysore Masala Dosa Made With Ragi Dosa Batter,99.69%,"[idli rice, ragi seed, white urad dal whole, m...",Karnataka,South Indian Breakfast,Vegetarian,[mustard],60.0
2528,Masala Dosa Recipe With Vegetable Filling Recipe,95.06%,"[dosa rice, black urad dal split, methi seed f...",South Indian,South Indian Breakfast,Vegetarian,unknown,40.0
8368,Masala Dosa Recipe With Vegetable Filling Recipe,95.0%,"[dosa rice, black urad dal split, methi seed f...",South Indian,South Indian Breakfast,Vegetarian,[mustard],40.0
2044,Sagu Masala Dosa Recipe,54.25%,"[white urad dal split, red matta rice, dosa ri...",Karnataka,South Indian Breakfast,Vegetarian,[mustard],55.0
7636,Sagu Masala Dosa Recipe,52.52%,"[white urad dal split, red matta rice, dosa ri...",Karnataka,South Indian Breakfast,Vegetarian,[mustard],55.0


Get recipe recommendation using best NMF model on Count vectorizer

In [9]:
recommended_recipes_nmf_count = get_recommendations(filtered_df, count_vectorizer, NMF) 
recommended_recipes_nmf_count

c:\Users\rogha\anaconda3\envs\Tasty_AI\lib\site-packages\sklearn\decomposition\_nmf.py:1759: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(
c:\Users\rogha\anaconda3\envs\Tasty_AI\lib\site-packages\sklearn\decomposition\_nmf.py:1759: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


Best Number of Topics: 9 with Coherence Score: 0.5616105461434897


,name,similarity_score,cleaned_ingredients,cuisine,course,diet,allergens,prep_time
5609,Mysore Masala Dosa Made With Ragi Dosa Batter,99.97%,"[idli rice, ragi seed, white urad dal whole, m...",Karnataka,South Indian Breakfast,Vegetarian,[mustard],60.0
2528,Masala Dosa Recipe With Vegetable Filling Recipe,97.3%,"[dosa rice, black urad dal split, methi seed f...",South Indian,South Indian Breakfast,Vegetarian,unknown,40.0
8368,Masala Dosa Recipe With Vegetable Filling Recipe,97.28%,"[dosa rice, black urad dal split, methi seed f...",South Indian,South Indian Breakfast,Vegetarian,[mustard],40.0
2546,Kollu Masala Dosa Recipe - High Protein Horse ...,80.2%,"[rice, horse gram dal kulith, methi seed fenug...",Konkan,South Indian Breakfast,Vegetarian,[mustard],960.0
8393,Kollu Masala Dosa Recipe - High Protein Horse ...,79.32%,"[rice, horse gram dal kulith, methi seed fenug...",Konkan,South Indian Breakfast,Vegetarian,[mustard],960.0


Get recipe recommendation using best SVD model on TF-IDF vectorizer

In [10]:
recommended_recipes_svd_tfidf = get_recommendations(filtered_df, tfidf_vectorizer, TruncatedSVD) 
recommended_recipes_svd_tfidf

Best Number of Topics: 6 with Coherence Score: 0.742839424532284


,name,similarity_score,cleaned_ingredients,cuisine,course,diet,allergens,prep_time
5609,Mysore Masala Dosa Made With Ragi Dosa Batter,99.78%,"[idli rice, ragi seed, white urad dal whole, m...",Karnataka,South Indian Breakfast,Vegetarian,[mustard],60.0
1238,Egg Roast Masala Dosa Recipe - With Moong Dal ...,97.08%,"[yellow moong dal split, idli rice, methi seed...",South Indian,South Indian Breakfast,Eggetarian,"[mustard, egg]",30.0
6428,Egg Roast Masala Dosa Recipe - With Moong Dal ...,96.86%,"[yellow moong dal split, idli rice, methi seed...",South Indian,South Indian Breakfast,Eggetarian,"[mustard, egg]",30.0
8393,Kollu Masala Dosa Recipe - High Protein Horse ...,87.56%,"[rice, horse gram dal kulith, methi seed fenug...",Konkan,South Indian Breakfast,Vegetarian,[mustard],960.0
2546,Kollu Masala Dosa Recipe - High Protein Horse ...,87.36%,"[rice, horse gram dal kulith, methi seed fenug...",Konkan,South Indian Breakfast,Vegetarian,[mustard],960.0


Get recipe recommendation using best SVD model on Count vectorizer

In [11]:
recommended_recipes_svd_count = get_recommendations(filtered_df, count_vectorizer, TruncatedSVD) 
recommended_recipes_svd_count

Best Number of Topics: 5 with Coherence Score: 0.6741790975105314


,name,similarity_score,cleaned_ingredients,cuisine,course,diet,allergens,prep_time
5609,Mysore Masala Dosa Made With Ragi Dosa Batter,99.94%,"[idli rice, ragi seed, white urad dal whole, m...",Karnataka,South Indian Breakfast,Vegetarian,[mustard],60.0
1238,Egg Roast Masala Dosa Recipe - With Moong Dal ...,95.08%,"[yellow moong dal split, idli rice, methi seed...",South Indian,South Indian Breakfast,Eggetarian,"[mustard, egg]",30.0
6428,Egg Roast Masala Dosa Recipe - With Moong Dal ...,94.96%,"[yellow moong dal split, idli rice, methi seed...",South Indian,South Indian Breakfast,Eggetarian,"[mustard, egg]",30.0
8393,Kollu Masala Dosa Recipe - High Protein Horse ...,94.07%,"[rice, horse gram dal kulith, methi seed fenug...",Konkan,South Indian Breakfast,Vegetarian,[mustard],960.0
2546,Kollu Masala Dosa Recipe - High Protein Horse ...,94.06%,"[rice, horse gram dal kulith, methi seed fenug...",Konkan,South Indian Breakfast,Vegetarian,[mustard],960.0


Calculate the best number of topics using coherence score for only filtered_df["processed_name"]

In [12]:
def compute_coherence_scores(filtered_df, vectorizer, model, min_topics=2, max_topics_limit = 30):
    coherence_scores = []
    
    # Transform texts to TF-IDF matrix
    vectorizer_matrix = vectorizer.fit_transform(filtered_df["processed_name"])
    
    # Tokenize each text in `texts` for Gensim's coherence calculation
    tokenized_texts = [text.split() for text in filtered_df["processed_name"]]
    dictionary = Dictionary(tokenized_texts)
    corpus = [dictionary.doc2bow(text) for text in tokenized_texts]

    max_topics = min(max_topics_limit, vectorizer_matrix.shape[0])
    num_topics_range = range(min_topics, max_topics + 1)
        
    # Iterate over each specified topic count
    for num_topics in num_topics_range :
        called_model = model(n_components=num_topics, random_state=42)
        called_matrix = called_model.fit_transform(vectorizer_matrix)
        
        # Extract top words for each topic and format as list of tokens
        feature_names = vectorizer.get_feature_names_out()
        topics = [[feature_names[i] for i in topic.argsort()[:-11:-1]] for topic in called_model.components_]
        
        tokenized_topics = [[word for word in topic] for topic in topics]# Coherence model requires a list of tokenized topics

        coherence_model = CoherenceModel(
            topics=tokenized_topics, 
            texts=tokenized_texts, 
            dictionary=dictionary, 
            coherence='c_v'
        )
        coherence_score = coherence_model.get_coherence()
        coherence_scores.append(coherence_score)
        
        #print(f"Number of Topics: {num_topics}, Coherence Score: {coherence_score}")

    # Find the best number of topics
    best_num_topics = num_topics_range[coherence_scores.index(max(coherence_scores))]
    print(f"Best Number of Topics: {best_num_topics} with Coherence Score: {max(coherence_scores)}")

    return best_num_topics

Train the recommendation model using topic models and vectorizers for only filtered_df["processed_name"]

In [13]:
def get_recommendations_name(filtered_df, vectorizer, model, num_recommendations=5): 

    best_num_topics = compute_coherence_scores(filtered_df, vectorizer, model)

    vectorizer_matrix = vectorizer.fit_transform(filtered_df["processed_name"])

    # Ensure all values in tfidf_matrix are non-negative and finite
    vectorizer_matrix = np.nan_to_num(vectorizer_matrix.toarray(), nan=0.0, posinf=0.0, neginf=0.0)

    # Train the NMF model with the optimal number of topics    
    called_model = model(n_components=best_num_topics, random_state=42)
    called_model.fit(vectorizer_matrix)
    topic_matrix = called_model.transform(vectorizer_matrix)  # Topic distribution for each recipe

    filtered_df = filtered_df.copy()

    # Add topic distributions to the filtered DataFrame
    for i in range(best_num_topics):
        filtered_df[f'topic_{i}'] = topic_matrix[:, i]

    # Locate the index of the dish within the filtered DataFrame
    dish_index = filtered_df.reset_index().index[0]  # Get the first match if there are multiple

    # Calculate cosine similarity for the topics
    cosine_similarities = cosine_similarity(topic_matrix)
    
    # Get indices of the most similar recipes
    similar_indices = cosine_similarities[dish_index].argsort()[-num_recommendations-1:-1][::-1]

    # Fetch recommended recipes
    recommended_recipes = filtered_df.iloc[similar_indices].copy()
    recommended_recipes["similarity_score"] = cosine_similarities[dish_index][similar_indices]

    # Scale and format similarity scores to percentages
    recommended_recipes['similarity_score'] = (recommended_recipes['similarity_score'] * 100).round(2).astype(str) + '%'

    return recommended_recipes[['name', 'similarity_score', 'cleaned_ingredients', 'cuisine', 'course', 'diet', 'allergens', 'prep_time']]

Compare recipes recommended by NMF and SVD models on Tf-IDF and Count vectorizers using only filtered_df["processed_name"]

In [14]:
name_recommended_recipes_nmf_tfidf = get_recommendations_name(filtered_df, tfidf_vectorizer, NMF) 
name_recommended_recipes_nmf_tfidf

c:\Users\rogha\anaconda3\envs\Tasty_AI\lib\site-packages\sklearn\decomposition\_nmf.py:1759: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(
c:\Users\rogha\anaconda3\envs\Tasty_AI\lib\site-packages\sklearn\decomposition\_nmf.py:136: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(res * 2)


Best Number of Topics: 23 with Coherence Score: 0.4680541887820066


,name,similarity_score,cleaned_ingredients,cuisine,course,diet,allergens,prep_time
5609,Mysore Masala Dosa Made With Ragi Dosa Batter,100.0%,"[idli rice, ragi seed, white urad dal whole, m...",Karnataka,South Indian Breakfast,Vegetarian,[mustard],60.0
1005,Mushroom Corn Masala Dosa Recipe Using Ragi Id...,0.16%,"[white urad dal whole, methi seed fenugreek se...",Fusion,South Indian Breakfast,Vegetarian,unknown,35.0
6039,Mushroom Corn Masala Dosa Recipe Using Ragi Id...,0.16%,"[white urad dal whole, methi seed fenugreek se...",Fusion,South Indian Breakfast,Vegetarian,[peanut],35.0
6333,Schezwan Masala Dosa Recipe Using Classic Idli...,0.05%,"[white urad dal whole, methi seed fenugreek se...",Fusion,Snack,Vegetarian,"[soy, sesame]",70.0
1189,Schezwan Masala Dosa Recipe Using Classic Idli...,0.05%,"[white urad dal whole, methi seed fenugreek se...",Fusion,Snack,Vegetarian,unknown,70.0


In [15]:
name_recommended_recipes_nmf_count = get_recommendations_name(filtered_df, count_vectorizer, NMF) 
name_recommended_recipes_nmf_count

c:\Users\rogha\anaconda3\envs\Tasty_AI\lib\site-packages\sklearn\decomposition\_nmf.py:1759: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(
c:\Users\rogha\anaconda3\envs\Tasty_AI\lib\site-packages\sklearn\decomposition\_nmf.py:1759: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(
c:\Users\rogha\anaconda3\envs\Tasty_AI\lib\site-packages\sklearn\decomposition\_nmf.py:1759: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


Best Number of Topics: 24 with Coherence Score: 0.4875853805016859


c:\Users\rogha\anaconda3\envs\Tasty_AI\lib\site-packages\sklearn\decomposition\_nmf.py:1759: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


,name,similarity_score,cleaned_ingredients,cuisine,course,diet,allergens,prep_time
5609,Mysore Masala Dosa Made With Ragi Dosa Batter,100.0%,"[idli rice, ragi seed, white urad dal whole, m...",Karnataka,South Indian Breakfast,Vegetarian,[mustard],60.0
8424,Onion Tomato Podi Masala Dosa Recipe,88.68%,"[idli dosa batter, onion, idli dosa podi milag...",South Indian,South Indian Breakfast,Vegetarian,[mustard],30.0
2568,Onion Tomato Podi Masala Dosa Recipe,88.68%,"[idli dosa batter, onion, idli dosa podi milag...",South Indian,South Indian Breakfast,Vegetarian,[mustard],30.0
2044,Sagu Masala Dosa Recipe,71.79%,"[white urad dal split, red matta rice, dosa ri...",Karnataka,South Indian Breakfast,Vegetarian,[mustard],55.0
7636,Sagu Masala Dosa Recipe,71.79%,"[white urad dal split, red matta rice, dosa ri...",Karnataka,South Indian Breakfast,Vegetarian,[mustard],55.0


In [16]:
name_recommended_recipes_svd_tfidf = get_recommendations_name(filtered_df, tfidf_vectorizer, TruncatedSVD) 
name_recommended_recipes_svd_tfidf

Best Number of Topics: 26 with Coherence Score: 0.5099815139081895


,name,similarity_score,cleaned_ingredients,cuisine,course,diet,allergens,prep_time
5609,Mysore Masala Dosa Made With Ragi Dosa Batter,100.0%,"[idli rice, ragi seed, white urad dal whole, m...",Karnataka,South Indian Breakfast,Vegetarian,[mustard],60.0
1005,Mushroom Corn Masala Dosa Recipe Using Ragi Id...,48.73%,"[white urad dal whole, methi seed fenugreek se...",Fusion,South Indian Breakfast,Vegetarian,unknown,35.0
6039,Mushroom Corn Masala Dosa Recipe Using Ragi Id...,48.73%,"[white urad dal whole, methi seed fenugreek se...",Fusion,South Indian Breakfast,Vegetarian,[peanut],35.0
6041,Kanchipuram Masala Dosa Recipe With Soya Dosa ...,31.28%,"[idli rice, soybean whole soya dal, white urad...",South Indian,South Indian Breakfast,Vegetarian,"[soy, mustard, soya]",390.0
1007,Kanchipuram Masala Dosa Recipe With Soya Dosa ...,31.28%,"[idli rice, soybean whole soya dal, white urad...",South Indian,South Indian Breakfast,Vegetarian,"[soy, mustard, soya]",390.0


In [17]:
name_recommended_recipes_svd_count = get_recommendations_name(filtered_df, count_vectorizer, TruncatedSVD) 
name_recommended_recipes_svd_count

Best Number of Topics: 26 with Coherence Score: 0.4997167416570156


,name,similarity_score,cleaned_ingredients,cuisine,course,diet,allergens,prep_time
5609,Mysore Masala Dosa Made With Ragi Dosa Batter,100.0%,"[idli rice, ragi seed, white urad dal whole, m...",Karnataka,South Indian Breakfast,Vegetarian,[mustard],60.0
6039,Mushroom Corn Masala Dosa Recipe Using Ragi Id...,78.26%,"[white urad dal whole, methi seed fenugreek se...",Fusion,South Indian Breakfast,Vegetarian,[peanut],35.0
1005,Mushroom Corn Masala Dosa Recipe Using Ragi Id...,78.26%,"[white urad dal whole, methi seed fenugreek se...",Fusion,South Indian Breakfast,Vegetarian,unknown,35.0
1007,Kanchipuram Masala Dosa Recipe With Soya Dosa ...,75.0%,"[idli rice, soybean whole soya dal, white urad...",South Indian,South Indian Breakfast,Vegetarian,"[soy, mustard, soya]",390.0
6041,Kanchipuram Masala Dosa Recipe With Soya Dosa ...,75.0%,"[idli rice, soybean whole soya dal, white urad...",South Indian,South Indian Breakfast,Vegetarian,"[soy, mustard, soya]",390.0
